Table 4.11.1: Housing situation of new presentations to treatment

In [1]:
from gssutils import *

if is_interactive():
    scraper = Scraper('https://www.gov.uk/government/collections/alcohol-and-drug-misuse-and-treatment-statistics')
    scraper.select_dataset(title=lambda x: x.startswith('Substance misuse treatment for adults'), latest=True)
    tabs = {tab.name: tab for tab in scraper.distribution(title=lambda x: x.startswith('Data tables')).as_databaker()}

In [2]:
tab = tabs['Table 4.11.1']

https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/752515/AdultSubstanceMisuseNDTMSDataTables2017-18.xlsx

In [3]:
cell = tab.filter('Housing situation')
cell.assert_one()
obs = tab.filter('Inconsistent/missing').shift(0,1).expand(RIGHT)
observations = tab.filter('n').fill(DOWN).is_not_blank().is_not_whitespace() - obs
Housingsituation = cell.expand(DOWN).is_not_blank().is_not_whitespace()
Clients = cell.expand(RIGHT).is_not_blank().is_not_whitespace()
Dimensions = [
            HDim(Housingsituation,'Treatment group',DIRECTLY,LEFT),
            HDim(Clients,'Clients in treatment',CLOSEST,LEFT),
            HDimConst('Measure Type','count-of-new-presentations-to-treatment'),
            HDimConst('Unit','new-presentations-to-treatment'),
            HDimConst('Period', 'gregorian-interval/2017-04-01T00:00:00/P1Y')
            ]

In [4]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
if is_interactive():
    savepreviewhtml(c1)

In [5]:
new_table = c1.topandas()

In [6]:
import numpy as np
new_table['OBS'].replace('', np.nan, inplace=True)
new_table.dropna(subset=['OBS'], inplace=True)
new_table.rename(columns={'OBS': 'Value'}, inplace=True)
new_table['Value'] = new_table['Value'].astype(int)
new_table['Value'] = new_table['Value'].map(lambda x:''
                                  if (x == '-')
                                  else int(x))

In [7]:
new_table['Basis of treatment'] = new_table['Treatment group'].map(
    lambda x: {
        'Housing situation':'housing-situation',
        'No problem':'housing-situation/no-problem',
        'Housing problem':'housing-situation/housing-problem',
        'Urgent housing problem (NFA)':'housing-situation/urgent-housing-problem-nfa',
        'Other':'housing-situation/other',
        'Inconsistent/missing':'housing-situation/inconsistent/missing',
        'Total':'housing-situation/total' }.get(x, x))

In [8]:
import urllib.request as request
import csv
import io
import requests
vrl="https://raw.githubusercontent.com/ONS-OpenData/ref_alcohol/master/codelists/substance-type.csv"
t=requests.get(vrl).content
g=pd.read_csv(io.StringIO(t.decode('utf-8')))
new_table = pd.merge(new_table, g, how = 'left', left_on = 'Clients in treatment', right_on = 'Label')
new_table.columns = ['Substance type' if x=='Notation' else x for x in new_table.columns]
new_table = new_table[['Period','Basis of treatment','Substance type','Measure Type','Value','Unit']]

In [9]:
new_table

,Period,Basis of treatment,Substance type,Measure Type,Value,Unit
0,gregorian-interval/2017-04-01T00:00:00/P1Y,housing-situation/no-problem,opiate,Count,27931,new presentations to treatment
1,gregorian-interval/2017-04-01T00:00:00/P1Y,housing-situation/no-problem,non-opiate-only,Count,13097,new presentations to treatment
2,gregorian-interval/2017-04-01T00:00:00/P1Y,housing-situation/no-problem,non-opiate-and-alcohol,Count,14514,new presentations to treatment
3,gregorian-interval/2017-04-01T00:00:00/P1Y,housing-situation/no-problem,alcohol-only,Count,44733,new presentations to treatment
4,gregorian-interval/2017-04-01T00:00:00/P1Y,housing-situation/no-problem,total,Count,100275,new presentations to treatment
5,gregorian-interval/2017-04-01T00:00:00/P1Y,housing-situation/housing-problem,opiate,Count,6272,new presentations to treatment
6,gregorian-interval/2017-04-01T00:00:00/P1Y,housing-situation/housing-problem,non-opiate-only,Count,1793,new presentations to treatment
7,gregorian-interval/2017-04-01T00:00:00/P1Y,housing-situation/housing-problem,non-opiate-and-alcohol,Count,2291,new presentations to treatment
8,gregorian-interval/2017-04-01T00:00:00/P1Y,housing-situation/housing-problem,alcohol-only,Count,3681,new presentations to treatment
9,gregorian-interval/2017-04-01T00:00:00/P1Y,housing-situation/housing-problem,total,Count,14037,new presentations to treatment
